In [1]:
#import all necessary libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
#load datasets
train = pd.read_csv('train_downsampled_10k.csv')
#test = pd.read_csv('test.csv')

Since the original challenge required that we use data from 2013 and 2014 to predict users’ preferred hotel clusters in 2015, we further split our new set of 10,000 users into training and testing sets by year; 2013 and part of 2014 for the new training set and the rest of 2014 for the testing set. For a 80/20 split, we will split our data set so that anything after September 2014 is in test_new and anything before is in train_new.


In [3]:
train_new = train[((train.year == 2013) | ((train.year == 2014) & (train.month < 10)))]
test_new = train[((train.year == 2014) & (train.month >= 10))]

In [4]:
print(train_new.shape)
print(test_new.shape)

(225332, 28)
(65081, 28)


## Baseline model

The most simple method that we could try (beyond just randomly selecting clusters for each user) was to always select the most common clusters among the training set.

In [5]:
top_five_clusters = list(train.hotel_cluster.value_counts().head().index)
print(top_five_clusters)

[91, 48, 41, 64, 65]


We can use the most common clusters as our first list of predictions for each row in the test subset.

In [6]:
preds = [top_five_clusters for i in range(test_new.shape[0])]

In [7]:
import ml_metrics as metrics
target_preds = [[i] for i in test_new['hotel_cluster']]
metrics.mapk(target_preds, preds, k=5)

0.05042383081595755

The above code gave us a list of the 5 most common clusters and the MAP@5 score for this method was .0504. This provides a baseline value for which to evaluate the other models.